# Importing necessary libraries

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras import backend as K
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.models import load_model
import numpy as np
import pandas as pd
import glob
import cv2
import csv
import os 
from os import walk

/Users/leannwoo/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Define two functions

getFish: takes in a filepath and reads image data using OpenCV. Resizes image to 150x150 in RGB. Returns the image pixel data, the classifications (eyespot, no eyespot), and the species name. Classification and species name are parsed from the image filepath.

load_kfold_data: takes in k number of folds and a list of image filespaths. Calls getFish to receive image data and appends image data and classification data to respective lists. Label data is converted to binary in order to run in function StratifiedKFold. Convert labels back to categorical. Returns folds, training data, and labels.

In [2]:
def getFish(filepath):
    img = cv2.imread(filepath)
    # standardize size for prediction step
    img = cv2.resize(img, (150, 150))
    img = img/255.0
    img = img.reshape((1,) + img.shape)
    
    if "NoVertStripes" in filepath:
        classification = "NoVertStripe"
    else:
        classification = "VertStripes"  
    
    _, _, species = filepath.partition('VertStripes/')
    
    #res = model.predict(img)
    return(img, classification, species)

def load_data_kfold(k, image_filepaths):
    
    #create lists for image training data and label training data
    x_train = []
    y_train = []
    
    #iterate through list of image filepaths
    #get pixel, label data and store in x,y
    #append x and y to x_train and y_train lists
    #convert classification to binary
    for i in range(len(image_filepaths)):
        x, y, _ = getFish(image_filepaths[i])
        x_train.append(x)
        y_train.append(y)
        if (y_train[i] == 'VertStripes'):
            y_train[i] = 0
        else:
            y_train[i] = 1
    
    #StratifiedKFold can only work in binary; convert labels to categorical afterwards
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(x_train, y_train))
    
    #convert labels back to categorical from binary
    y_train = to_categorical(y_train)
    
    return folds, x_train, y_train

# Running load_data_kfold

Load list of image filepaths into "fish_paths" by changing corresponding Dropbox paths  
Use kfold function to get 10 folds, the training data, and the trianing labels.

**each folder must contain at least 10 values**
**check with liz to see if we can add 3 images to the no stripes folder**

In [3]:
##CHANGE THESE ACCORDING TO YOUR PATH!!
stripe_path = "/Users/leannwoo/Dropbox/machine_learning_fishes/image_classifiers/Vertical_stripes/VertStripes/"
nostripe_path = "/Users/leannwoo/Dropbox/machine_learning_fishes/image_classifiers/Vertical_stripes/NoVertStripes/"

fish_paths = []

for (dirpath, dirnames, filenames) in walk(stripe_path):
    for i in range(len(filenames)):
        stripe_img = dirpath + filenames[i]
        if "Store" not in stripe_img:  
            fish_paths.append(stripe_img)
    break

for (dirpath, dirnames, filenames) in walk(nostripe_path):
    for i in range(len(filenames)):
        nostripe_img = dirpath + filenames[i]
        if "Store" not in nostripe_img:
            fish_paths.append(nostripe_img)
    break

k = 10
folds, x_train, y_train = load_data_kfold(k, fish_paths)

# Defining CNN

In [4]:
# dimensions of our images.
img_width, img_height = 150, 150

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Creating distorted images generator to create more training images

In [5]:
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 10
                        )

# Running training set using 10-fold cross validation

Now using the training data to train the model that was previously defined with a batch size of 20 and 30 epochs per fold  
  
Training and validation indices come from folds, which was previously defined when we ran load_data_kfold

In [6]:
batch_size = 40

for j, (train_idx, val_idx) in enumerate(folds):
    
    #Creating lists for training and validation batch sets
    X_train_cv = []
    y_train_cv = []
    X_valid_cv = []
    y_valid_cv = []
    
    for i in range(len(train_idx)):
        X_train_cv.append(x_train[train_idx[i]])
        y_train_cv.append(y_train[train_idx[i]])
    for i in range(len(val_idx)):
        X_valid_cv.append(x_train[val_idx[i]])
        y_valid_cv.append(y_train[val_idx[i]])
    
    X_train_cv = np.array(X_train_cv)
    X_train_cv = np.squeeze(X_train_cv, axis = 1)
    
    X_valid_cv = np.array(X_valid_cv)
    X_valid_cv = np.squeeze(X_valid_cv, axis = 1)
    
    y_train_cv = np.array(y_train_cv)
    y_valid_cv = np.array(y_valid_cv)
    
    print '\nFold ',j
    #name_weights = "final_model_fold" + str(j) + "_weights.h5"
    generator = gen.flow(X_train_cv, y_train_cv, batch_size = batch_size)
    model.fit_generator(
                generator,
                steps_per_epoch=len(X_train_cv)/batch_size,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data = (X_valid_cv, y_valid_cv))
#     model.save_weights('first_try.h5')
    model.save('stripe_model.h5')
    print(model.evaluate(X_valid_cv, y_valid_cv))


Fold  0
Epoch 1/100
4/4 [==============================] - 7s 2s/step - loss: 0.2813 - acc: 0.5501 - val_loss: 0.2473 - val_acc: 0.5000
Epoch 2/100
4/4 [==============================] - 7s 2s/step - loss: 0.2484 - acc: 0.6188 - val_loss: 0.2392 - val_acc: 0.5500
Epoch 3/100
4/4 [==============================] - 6s 1s/step - loss: 0.2481 - acc: 0.6127 - val_loss: 0.2181 - val_acc: 0.7500
Epoch 4/100
4/4 [==============================] - 6s 2s/step - loss: 0.2168 - acc: 0.6725 - val_loss: 0.1754 - val_acc: 0.8000
Epoch 5/100
4/4 [==============================] - 6s 1s/step - loss: 0.1905 - acc: 0.7571 - val_loss: 0.1634 - val_acc: 0.9000
Epoch 6/100
4/4 [==============================] - 6s 1s/step - loss: 0.2046 - acc: 0.6495 - val_loss: 0.1449 - val_acc: 0.8000
Epoch 7/100
4/4 [==============================] - 6s 1s/step - loss: 0.1316 - acc: 0.8418 - val_loss: 0.1394 - val_acc: 0.7500
Epoch 8/100
4/4 [==============================] - 6s 2s/step - loss: 0.1604 - acc: 0.7903 - va

4/4 [==============================] - 10s 2s/step - loss: 0.0475 - acc: 0.9438 - val_loss: 0.1417 - val_acc: 0.8500
Epoch 65/100
4/4 [==============================] - 7s 2s/step - loss: 0.0753 - acc: 0.9071 - val_loss: 0.2379 - val_acc: 0.7500
Epoch 66/100
4/4 [==============================] - 6s 2s/step - loss: 0.1041 - acc: 0.8841 - val_loss: 0.1297 - val_acc: 0.8500
Epoch 67/100
4/4 [==============================] - 7s 2s/step - loss: 0.0427 - acc: 0.9438 - val_loss: 0.1225 - val_acc: 0.8500
Epoch 68/100
4/4 [==============================] - 6s 1s/step - loss: 0.0310 - acc: 0.9799 - val_loss: 0.1439 - val_acc: 0.8500
Epoch 69/100
4/4 [==============================] - 7s 2s/step - loss: 0.0602 - acc: 0.9430 - val_loss: 0.1226 - val_acc: 0.8500
Epoch 70/100
4/4 [==============================] - 8s 2s/step - loss: 0.0490 - acc: 0.9375 - val_loss: 0.1161 - val_acc: 0.8500
Epoch 71/100
4/4 [==============================] - 8s 2s/step - loss: 0.0236 - acc: 0.9742 - val_loss: 0.133

Epoch 27/100
4/4 [==============================] - 7s 2s/step - loss: 0.0285 - acc: 0.9625 - val_loss: 0.0067 - val_acc: 1.0000
Epoch 28/100
4/4 [==============================] - 6s 1s/step - loss: 0.0492 - acc: 0.9485 - val_loss: 0.0804 - val_acc: 0.9000
Epoch 29/100
4/4 [==============================] - 6s 1s/step - loss: 0.0559 - acc: 0.9246 - val_loss: 0.0180 - val_acc: 1.0000
Epoch 30/100
4/4 [==============================] - 6s 1s/step - loss: 0.0273 - acc: 0.9614 - val_loss: 0.0146 - val_acc: 1.0000
Epoch 31/100
4/4 [==============================] - 6s 1s/step - loss: 0.0196 - acc: 0.9678 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 32/100
4/4 [==============================] - 7s 2s/step - loss: 0.0455 - acc: 0.9438 - val_loss: 0.0027 - val_acc: 1.0000
Epoch 33/100
4/4 [==============================] - 6s 1s/step - loss: 0.0735 - acc: 0.9043 - val_loss: 0.0218 - val_acc: 0.9500
Epoch 34/100
4/4 [==============================] - 6s 1s/step - loss: 0.0142 - acc: 0.9871 - val

Epoch 90/100
4/4 [==============================] - 6s 1s/step - loss: 0.0431 - acc: 0.9558 - val_loss: 6.0271e-05 - val_acc: 1.0000
Epoch 91/100
4/4 [==============================] - 6s 2s/step - loss: 0.0342 - acc: 0.9500 - val_loss: 0.0013 - val_acc: 1.0000
Epoch 92/100
4/4 [==============================] - 5s 1s/step - loss: 0.0333 - acc: 0.9622 - val_loss: 0.0017 - val_acc: 1.0000
Epoch 93/100
4/4 [==============================] - 7s 2s/step - loss: 0.0457 - acc: 0.9250 - val_loss: 0.0053 - val_acc: 1.0000
Epoch 94/100
4/4 [==============================] - 6s 1s/step - loss: 0.0328 - acc: 0.9488 - val_loss: 0.0270 - val_acc: 0.9500
Epoch 95/100
4/4 [==============================] - 7s 2s/step - loss: 0.0442 - acc: 0.9438 - val_loss: 3.4022e-04 - val_acc: 1.0000
Epoch 96/100
4/4 [==============================] - 6s 2s/step - loss: 0.0223 - acc: 0.9750 - val_loss: 1.9100e-04 - val_acc: 1.0000
Epoch 97/100
4/4 [==============================] - 6s 1s/step - loss: 0.0123 - acc: 

Epoch 52/100
4/4 [==============================] - 6s 1s/step - loss: 0.0168 - acc: 0.9880 - val_loss: 1.4608e-04 - val_acc: 1.0000
Epoch 53/100
4/4 [==============================] - 7s 2s/step - loss: 0.0253 - acc: 0.9563 - val_loss: 1.2301e-04 - val_acc: 1.0000
Epoch 54/100
4/4 [==============================] - 5s 1s/step - loss: 0.0500 - acc: 0.9366 - val_loss: 3.5561e-04 - val_acc: 1.0000
Epoch 55/100
4/4 [==============================] - 7s 2s/step - loss: 0.0122 - acc: 0.9875 - val_loss: 0.0421 - val_acc: 0.9500
Epoch 56/100
4/4 [==============================] - 6s 1s/step - loss: 0.0202 - acc: 0.9807 - val_loss: 0.0057 - val_acc: 1.0000
Epoch 57/100
4/4 [==============================] - 6s 1s/step - loss: 0.0248 - acc: 0.9742 - val_loss: 1.4500e-04 - val_acc: 1.0000
Epoch 58/100
4/4 [==============================] - 6s 1s/step - loss: 0.0014 - acc: 1.0000 - val_loss: 8.5174e-05 - val_acc: 1.0000
Epoch 59/100
4/4 [==============================] - 6s 1s/step - loss: 0.0191

...

4/4 [==============================] - 7s 2s/step - loss: 0.0063 - acc: 0.9936 - val_loss: 7.8644e-04 - val_acc: 1.0000
Epoch 45/100
4/4 [==============================] - 7s 2s/step - loss: 0.0063 - acc: 0.9936 - val_loss: 3.0813e-04 - val_acc: 1.0000
Epoch 46/100
4/4 [==============================] - 6s 2s/step - loss: 0.0017 - acc: 1.0000 - val_loss: 2.5472e-06 - val_acc: 1.0000
Epoch 47/100
4/4 [==============================] - 7s 2s/step - loss: 0.0013 - acc: 1.0000 - val_loss: 4.3389e-08 - val_acc: 1.0000
Epoch 48/100
4/4 [==============================] - 5s 1s/step - loss: 0.0063 - acc: 0.9933 - val_loss: 5.2324e-07 - val_acc: 1.0000
Epoch 49/100
4/4 [==============================] - 7s 2s/step - loss: 0.0061 - acc: 0.9936 - val_loss: 3.1280e-06 - val_acc: 1.0000
Epoch 50/100
4/4 [==============================] - 7s 2s/step - loss: 0.0069 - acc: 0.9938 - val_loss: 1.9693e-07 - val_acc: 1.0000
Epoch 51/100
4/4 [==============================] - 6s 2s/step - loss: 0.0202 - ac

4/4 [==============================] - 7s 2s/step - loss: 0.0090 - acc: 0.9875 - val_loss: 1.8901e-09 - val_acc: 1.0000
Epoch 6/100
4/4 [==============================] - 6s 2s/step - loss: 0.0014 - acc: 1.0000 - val_loss: 8.7575e-13 - val_acc: 1.0000
Epoch 7/100
4/4 [==============================] - 5s 1s/step - loss: 2.6512e-06 - acc: 1.0000 - val_loss: 7.6777e-13 - val_acc: 1.0000
Epoch 8/100
4/4 [==============================] - 7s 2s/step - loss: 0.0228 - acc: 0.9750 - val_loss: 2.3512e-07 - val_acc: 1.0000
Epoch 9/100
4/4 [==============================] - 6s 1s/step - loss: 0.0176 - acc: 0.9816 - val_loss: 1.8652e-14 - val_acc: 1.0000
Epoch 10/100
4/4 [==============================] - 6s 1s/step - loss: 8.1053e-04 - acc: 1.0000 - val_loss: 1.1120e-06 - val_acc: 1.0000
Epoch 11/100
4/4 [==============================] - 6s 1s/step - loss: 0.0026 - acc: 1.0000 - val_loss: 4.0959e-14 - val_acc: 1.0000
Epoch 12/100
4/4 [==============================] - 7s 2s/step - loss: 0.0070 

4/4 [==============================] - 7s 2s/step - loss: 0.0068 - acc: 0.9938 - val_loss: 4.9631e-08 - val_acc: 1.0000
Epoch 67/100
4/4 [==============================] - 7s 2s/step - loss: 2.6438e-05 - acc: 1.0000 - val_loss: 3.6418e-08 - val_acc: 1.0000
Epoch 68/100
4/4 [==============================] - 5s 1s/step - loss: 0.0066 - acc: 0.9933 - val_loss: 6.2943e-08 - val_acc: 1.0000
Epoch 69/100
4/4 [==============================] - 7s 2s/step - loss: 2.5687e-04 - acc: 1.0000 - val_loss: 6.2776e-09 - val_acc: 1.0000
Epoch 70/100
4/4 [==============================] - 7s 2s/step - loss: 0.0137 - acc: 0.9871 - val_loss: 2.8986e-11 - val_acc: 1.0000
Epoch 71/100
4/4 [==============================] - 7s 2s/step - loss: 0.0205 - acc: 0.9687 - val_loss: 2.2321e-06 - val_acc: 1.0000
Epoch 72/100
4/4 [==============================] - 7s 2s/step - loss: 0.0231 - acc: 0.9623 - val_loss: 1.2685e-14 - val_acc: 1.0000
Epoch 73/100
4/4 [==============================] - 8s 2s/step - loss: 0.0

Epoch 27/100
4/4 [==============================] - 14s 3s/step - loss: 3.6667e-04 - acc: 1.0000 - val_loss: 5.6810e-12 - val_acc: 1.0000
Epoch 28/100
4/4 [==============================] - 12s 3s/step - loss: 1.7109e-04 - acc: 1.0000 - val_loss: 9.0370e-12 - val_acc: 1.0000
Epoch 29/100
4/4 [==============================] - 7s 2s/step - loss: 1.4798e-06 - acc: 1.0000 - val_loss: 9.7109e-12 - val_acc: 1.0000
Epoch 30/100
4/4 [==============================] - 8s 2s/step - loss: 5.5872e-08 - acc: 1.0000 - val_loss: 9.4497e-12 - val_acc: 1.0000
Epoch 31/100
4/4 [==============================] - 7s 2s/step - loss: 0.0067 - acc: 0.9936 - val_loss: 4.7284e-11 - val_acc: 1.0000
Epoch 32/100
4/4 [==============================] - 6s 2s/step - loss: 0.0072 - acc: 0.9936 - val_loss: 0.0451 - val_acc: 0.9474
Epoch 33/100
4/4 [==============================] - 9s 2s/step - loss: 0.0187 - acc: 0.9813 - val_loss: 0.0440 - val_acc: 0.9474
Epoch 34/100
4/4 [==============================] - 6s 2s/s

4/4 [==============================] - 7s 2s/step - loss: 0.0046 - acc: 0.9938 - val_loss: 1.2702e-04 - val_acc: 1.0000
Epoch 89/100
4/4 [==============================] - 6s 2s/step - loss: 9.2377e-04 - acc: 1.0000 - val_loss: 0.0526 - val_acc: 0.9474
Epoch 90/100
4/4 [==============================] - 6s 1s/step - loss: 0.0139 - acc: 0.9871 - val_loss: 1.6728e-06 - val_acc: 1.0000
Epoch 91/100
4/4 [==============================] - 6s 1s/step - loss: 0.0050 - acc: 0.9936 - val_loss: 6.2383e-18 - val_acc: 1.0000
Epoch 92/100
4/4 [==============================] - 7s 2s/step - loss: 1.4026e-05 - acc: 1.0000 - val_loss: 9.3780e-18 - val_acc: 1.0000
Epoch 93/100
4/4 [==============================] - 6s 2s/step - loss: 7.4736e-09 - acc: 1.0000 - val_loss: 9.3740e-18 - val_acc: 1.0000
Epoch 94/100
4/4 [==============================] - 7s 2s/step - loss: 0.0128 - acc: 0.9875 - val_loss: 3.4041e-18 - val_acc: 1.0000
Epoch 95/100
4/4 [==============================] - 7s 2s/step - loss: 0.0

# Predicting

Define prediction function, takes in a model and a list of returned data from getFish function. Returns the model's prediction, the correct classification, the prediction weights, and the fish family/species.

In [10]:
def predict(model, get_fish_output):
    
    img, correct_class, fam = get_fish_output
    
    prediction_weights = model.predict(img)
    prediction = model.predict_classes(img)
    if (prediction[0] == 0):
        prediction = "eyespot"
    else:
        prediction = "noeyespot"

    #print "Model classification:",prediction,"\nCorrect classification:", correct_class,"\nPrediction wieghts:",prediction_weights,"\nType:",fam,"\n"
    return(prediction, correct_class, prediction_weights, fam)

# Writing the model's classification data to CSV

Loads model in "mod"  
Writes the model classification, correct classification, prediction weights, and species/family to a csv files  
Calculates data for confusion matrix

In [8]:
mod = load_model("stripe_model.h5")

with open('classification_data.csv', 'wb') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Model Classification', 'Correct Classification', 'Prediction Weights', 'Type'])

    true_eye=0
    true_noeye=0
    false_eye=0
    false_noeye=0

    for i in range(len(fish_paths)):
        info = predict(mod, getFish(fish_paths[i]))
        filewriter.writerow([info[0], info[1], info[2], info[3]])

        #computing info for confusion matrix
        if (info[0] == info[1]):
            if (info[0] == 'eyespot'):
                true_eye += 1
            else:
                true_noeye += 1
        else:
            if (info[0] == 'eyespot'):
                false_eye += 1
            else:
                false_noeye += 1

# Prints confusion matrix

In [9]:
data = {'Eyespots':[true_eye, false_noeye], 'No Eyespots':[false_eye, true_noeye] }
confusion_matrix = pd.DataFrame(data = data)
confusion_matrix = confusion_matrix.rename({0:'Eyespots (Model Output)', 1:'No Eyespots (Model Output)'},axis='index')
confusion_matrix

,Eyespots,No Eyespots
Eyespots (Model Output),38,1
No Eyespots (Model Output),2,39


# Running classifier on Chaets

Right now, the output in "info" is a little messy, but you can see the prediction and prediction weights. The function will automatically assume that "eyespot" is the correct classification due to how the function is written. It will also not list what the species name is. It's pretty messy for now, but I'll work on cleaning it up.

In [15]:
chaet_path = "/Users/leannwoo/Dropbox/machine_learning_fishes/randall/Chaetodontidae/"

chaets = []

for (dirpath, dirnames, filenames) in walk(chaet_path):
    for i in range(len(filenames)):
        img = dirpath + filenames[i]
        if "Store" not in img:  
            chaets.append(img)
    break
    
mod = load_model("second_try.h5")
info = []

for i in range(len(chaets)):
    info.append(predict(mod, getFish(chaets[i])))
    
info

[('noeyespot',
  'eyespot',
  array([[1.5743046e-06, 9.9999642e-01]], dtype=float32),
  ''),
 ('noeyespot',
  'eyespot',
  array([[5.3404574e-04, 9.9945408e-01]], dtype=float32),
  ''),
 ('noeyespot',
  'eyespot',
  array([[0.00922593, 0.99053675]], dtype=float32),
  ''),
 ('eyespot', 'eyespot', array([[0.6216251 , 0.54611564]], dtype=float32), ''),
 ('eyespot', 'eyespot', array([[0.96949995, 0.03259671]], dtype=float32), ''),
 ('noeyespot',
  'eyespot',
  array([[6.2341300e-05, 9.9996173e-01]], dtype=float32),
  ''),
 ('noeyespot',
  'eyespot',
  array([[0.00692909, 0.9931799 ]], dtype=float32),
  ''),
 ('noeyespot',
  'eyespot',
  array([[4.5743766e-07, 9.9999857e-01]], dtype=float32),
  ''),
 ('noeyespot',
  'eyespot',
  array([[8.772105e-07, 9.999994e-01]], dtype=float32),
  ''),
 ('eyespot',
  'eyespot',
  array([[9.9934715e-01, 8.1560889e-04]], dtype=float32),
  ''),
 ('eyespot', 'eyespot', array([[0.64428985, 0.3350438 ]], dtype=float32), ''),
 ('noeyespot',
  'eyespot',
  array